In [35]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [36]:
# Data can be downloaded at http://pannous.net/spoken_numbers.tar

In [37]:
SEED = 2017
DATA_DIR = 'C:/Users/xianl_pmrkzzf/Desktop/tensorflow-speech-recognition-master/spoken_numbers_pcm/'
print(DATA_DIR)

C:/Users/xianl_pmrkzzf/Desktop/tensorflow-speech-recognition-master/spoken_numbers_pcm/


In [61]:
files = glob.glob(DATA_DIR + "*.wav")
print(type(files[0]))
print(files[0])
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

<class 'str'>
C:/Users/xianl_pmrkzzf/Desktop/tensorflow-speech-recognition-master/spoken_numbers_pcm\0_Agnes_100.wav
# Training examples: 1920
# Validation examples: 480


In [56]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('\\')[-1].split('_')[1]
    if label not in labels:
        labels.append(label)
print(len(labels))

15


In [40]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [41]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [42]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [43]:
learning_rate = 0.001
batch_size = 64
n_epochs = 50
dropout = 0.5

input_shape = (n_features, max_length)
steps_per_epoch = 50

In [44]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

In [45]:
opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 20, 256)           345088    
_________________________________________________________________
flatten_2 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               655488    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 15)                1935      
Total params: 1,002,511
Trainable params: 1,002,511
Non-trainable params: 0
_________________________________________________________________


In [46]:
callbacks = [ModelCheckpoint('checkpoints/voice_recognition_best_model_{epoch:02d}.hdf5', save_best_only=True),
            EarlyStopping(monitor='val_acc', patience=2)]
#(callbacks[1])

In [48]:
history = model.fit_generator(
 generator=batch_generator(X_train, batch_size),
 steps_per_epoch=steps_per_epoch,
 epochs=n_epochs,
 verbose=1,
 validation_data=batch_generator(X_val, 32),
 validation_steps=5,
 callbacks=callbacks
 )


Epoch 1/50
50/50 [==============================] - 74s 1s/step - loss: 0.0000e+00 - accuracy: 0.0456 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50


D:\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50/50 [==============================] - 79s 2s/step - loss: 0.0000e+00 - accuracy: 0.0494 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/50
50/50 [==============================] - 82s 2s/step - loss: 0.0000e+00 - accuracy: 0.0566 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/50
50/50 [==============================] - 78s 2s/step - loss: 0.0000e+00 - accuracy: 0.0556 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/50
50/50 [==============================] - 80s 2s/step - loss: 0.0000e+00 - accuracy: 0.0609 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/50
50/50 [==============================] - 77s 2s/step - loss: 0.0000e+00 - accuracy: 0.0591 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/50
50/50 [==============================] - 78s 2s/step - loss: 0.0000e+00 - accuracy: 0.0594 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/50
50/50 [==============================] - 83s 2s/step - loss: 0.0000e+00 - accuracy: 0.05

In [54]:
history

In [ ]:
model.save